In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import time
import pandas as pd
from intermediates import *

In [2]:
import lsst.sims.maf.slicers as slicers
from lsst.obs.lsstSim import LsstSimMapper
from lsst.sims.utils import ObservationMetaData
from lsst.sims.coordUtils import chipNameFromRaDec

In [3]:
#help(ObservationMetaData)
#help(chipNameFromRaDec)

In [4]:
dbpath= '/Volumes/Transcend/minion_1016_sqlite.db'
#simdata= getSimData(dbpath, 'r')   # get the OpSim columns

In [5]:
def findChips(dpbath, focusDither, focusID, 
              filterBand= 'r', disc= True):
    
    FOV_radius= 0.0305
    nside= 512
    printProgress('Getting simData ... ', highlight= True)
    simdata= getSimData(dbpath, filterBand, ['expDate', 'obsHistID'])
    
    printProgress('Finding region pixels ... ', highlight= True)
    centralRA, centralDec, regionPixels= findRegionPixels(focusID, simdata,
                                                          nside, disc, FOV_radius)
    totPixels= len(regionPixels)
    printProgress('Total number of pixels: %d'%(totPixels))
                  
    # set up the slicer
    hpSlicer= slicers.HealpixSlicer(nside= nside)
    hpSlicer.setupSlicer(simdata)    # need to slice data?
    
    camera = LsstSimMapper().camera
    chipNames= []
    obsIDs= []
    expDates= []
    fIDs= []
    for p, pixel in enumerate(regionPixels):
        pixRA, pixDec= hpSlicer._pix2radec(pixel)    # radians returned    
        indObsInPixel = hpSlicer._sliceSimData(pixel)
        
        for index in indObsInPixel['idxs']:
            # get data
            # for identifying each visit
            expDate= simdata[index]['expDate']
            obsID= simdata[index]['obsHistID']
            fID= simdata[index]['fieldID']
            
            # for chip finding
            if (focusDither=='NoDither'):
                pointingRA= np.degrees(simdata[index]['fieldRA'])   # need in degres
                pointingDec= np.degrees(simdata[index]['fieldDec']) # need in degres
            else:
                pointingRA= np.degrees(simdata[index]['ditheredRA']) # need in degres
                pointingDec= np.degrees(simdata[index]['ditheredDec']) # need in degres
            rotSkyPos= np.degrees(simdata[index]['rotSkyPos']) # need in degres
            expMJD= simdata[index]['expMJD']
            
            # set up for the finding the chips
            obs = ObservationMetaData(pointingRA= pointingRA, pointingDec= pointingDec,
                                      rotSkyPos= rotSkyPos, mjd= expMJD)
            chipsInVisit= chipNameFromRaDec(np.degrees(pixRA), np.degrees(pixDec),
                                            camera=camera, obs_metadata=obs)
            if chipsInVisit is not None:
                obsIDs.append(obsID)
                expDates.append(expDate)
                chipNames.append(chipsInVisit)
                fIDs.append(fID)
        percentDone= 100.*(p+1)/totPixels
        
        ##if (percentDone>1.) and (percentDone%5.<1.):
        #    printProgress('Percent pixels done: %f'%(percentDone))
        if (percentDone>1.):
            return [np.array(obsIDs), np.array(expDates), np.array(fIDs), np.array(chipNames)]
    #return [np.array(obsIDs), np.array(expDates), np.array(fIDs), np.array(chipNames)]

In [6]:
startTime= time.time()
obsIDs, expDates, fIDs, chipNames= findChips(dbpath, '', 1447,
                                              filterBand= 'r', disc= True)
printProgress('Time Taken: %f'%(time.time()-startTime))


#############################################
## Getting simData ... 

#############################################
## Finding region pixels ... 
#
## Total number of pixels: 4568
Healpix slicer using NSIDE=512, approximate resolution 6.870973 arcminutes


/Users/Humna/lsst/DarwinX86/sims_utils/12.0-13-g6534730/python/lsst/sims/utils/ModifiedJulianDate.py:188: UTCtoUT1Warning: UTC is outside of IERS table for UT1-UTC.
Returning UT1 = UTC for lack of a better idea
This warning was caused by calling ModifiedJulianDate.dut1

  category=UTCtoUT1Warning)


#
## Time Taken: 202.260281


In [7]:
pd.DataFrame([obsIDs, expDates, fIDs, chipNames], ['obsHistID', 'expDate', 'fIDs', 'Chips'])

,0,1,2,3,4,5,6,7,8,9,...,7649,7650,7651,7652,7653,7654,7655,7656,7657,7658
obsHistID,2168708,1499774,1993954,456925,1993992,2182701,1299264,1700145,1499816,1422764,...,482282,703640,1515549,300271,1761905,1256066,2383159,1680321,943346,2184718
expDate,276072453,192253838,254460584,57044790,254462202,277800203,165286880,219217178,192255814,181896202,...,60751097,89616760,194067283,36381717,226466509,160367661,306317836,216618764,119340460,278059191
fIDs,1564,1568,1568,1568,1568,1568,1568,1568,1568,1568,...,1564,1564,1564,1564,1564,1564,1564,1564,1564,1564
Chips,"R:2,1 S:0,2","R:2,1 S:2,2","R:0,2 S:0,0","R:1,0 S:1,2","R:0,1 S:1,0","R:1,4 S:0,0","R:0,1 S:1,2","R:1,3 S:1,1","R:2,1 S:2,2","R:2,4 S:2,1",...,"R:1,3 S:1,1","R:3,1 S:0,1","R:2,3 S:2,0","R:0,3 S:2,0","R:3,0 S:2,2","R:4,3 S:0,1","R:2,0 S:0,2","R:2,0 S:1,1","R:2,2 S:0,0","R:3,3 S:2,0"


In [8]:
print len(chipNames)
print len(np.unique(chipNames))
print len(np.unique(expDates))
print len(np.unique(obsIDs))

7659
205
400
400


In [9]:
obsIDsList, expDatesList, fIDsList, chipNamesList= [], [], [], []

for obs in np.unique(obsIDs):
    obsIDsList.append(obs)
    ind= np.where(obsIDs==obs)[0]
    
    expDatesList.append(np.unique(expDates[ind]))
    
    fIDsList.append(np.unique(fIDs[ind]))
    chipNamesList.append(np.unique(chipNames[ind]))

In [10]:
len(obsIDsList)

400

In [11]:
pd.DataFrame([obsIDsList, expDatesList, fIDsList, chipNamesList], ['obsHistID', 'expDate', 'fIDs', 'Chips'])

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
obsHistID,193219,193220,194072,194125,201791,201794,201831,201835,202705,202745,...,2418912,2418926,2422464,2422465,2422508,2422509,2440030,2440031,2440067,2440068
expDate,[22409391],[22409429],[22494327],[22496709],[23612696],[23612812],[23614579],[23614784],[23702758],[23704330],...,[311321168],[311321738],[311834154],[311834193],[311836104],[311836142],[314252832],[314252871],[314254567],[314254614]
fIDs,[1564],[1568],[1568],[1568],[1568],[1564],[1568],[1564],[1564],[1564],...,[1568],[1564],[1568],[1564],[1568],[1564],[1564],[1568],[1564],[1568]
Chips,"[R:2,0 S:2,0]","[R:1,1 S:2,2, R:1,2 S:1,1, R:1,2 S:2,0, R:1,2 ...","[R:1,1 S:2,2, R:1,2 S:1,2, R:1,2 S:2,0, R:1,2 ...","[R:1,1 S:2,2, R:1,2 S:1,2, R:1,2 S:2,0, R:1,2 ...","[R:0,1 S:1,2, R:0,1 S:2,2, R:0,2 S:2,0, R:1,2 ...","[R:0,2 S:0,0, R:0,2 S:0,1, R:0,2 S:1,1]","[R:0,1 S:1,1, R:0,1 S:2,1, R:0,1 S:2,2, R:1,1 ...","[R:0,1 S:0,2, R:0,2 S:0,0, R:0,2 S:1,0]","[R:3,0 S:2,2, R:3,1 S:2,0, R:4,0 S:0,2,A, R:4,...","[R:3,0 S:2,2, R:3,1 S:1,0, R:4,0 S:0,1, R:4,0 ...",...,"[R:0,3 S:1,2, R:0,3 S:2,2, R:0,4 S:1,0, R:0,4 ...","[R:0,3 S:2,0, R:1,2 S:1,2, R:1,2 S:2,1, R:1,2 ...","[R:3,1 S:1,1, R:3,1 S:1,2, R:3,1 S:2,0, R:3,1 ...","[R:2,0 S:2,0, R:2,0 S:2,1, R:2,0 S:2,2, R:3,0 ...","[R:3,1 S:1,0, R:3,1 S:1,1, R:3,1 S:1,2, R:3,2 ...","[R:2,0 S:1,0, R:2,0 S:2,0, R:2,0 S:2,1, R:2,0 ...","[R:1,0 S:2,0, R:2,0 S:0,0, R:2,0 S:0,1, R:2,0 ...","[R:3,0 S:0,1, R:3,0 S:0,2, R:3,0 S:1,2, R:3,1 ...","[R:2,0 S:2,0, R:2,0 S:2,1, R:3,0 S:0,2, R:3,1 ...","[R:3,0 S:0,1, R:3,0 S:1,1, R:3,0 S:1,2, R:3,1 ..."


In [12]:
import pickle

dataToSave = {'obsHistID': obsIDsList, 'expDate': expDatesList, 'fIDs': fIDsList, 'chipNames': chipNamesList}

with open('chipPerVisitData.pickle', 'wb') as handle:
    pickle.dump(dataToSave, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
#with open('chipPerVisitData.pickle', 'rb') as handle:
#      savedData= pickle.load(handle)
